In [92]:
# API接続設定のファイルを読み込む
import configparser

In [93]:
# レート取得のために必要なライブラリ
from oandapyV20 import API
import oandapyV20.endpoints.instruments as instruments
import pandas as pd
import datetime
from datetime import datetime, timedelta

In [94]:
# 設定情報を"config_v1.txt"から読み込む
config = configparser.ConfigParser()
config.read('config_v1.txt') # パスの指定が必要です
account_id = config['oanda']['account_id']
api_key = config['oanda']['api_key']
api = API(access_token=api_key)

In [95]:
# 取得したい時間範囲を指定
params = {
    "alignmentTimezone": "Japan", # 時間を日本時間に
    "count": 1000, 
    "granularity": "M1" # 1分間隔
}

In [96]:
# APIへ過去データをリクエスト
r = instruments.InstrumentsCandles(instrument="USD_JPY", params=params)
api.request(r)

{'instrument': 'USD_JPY',
 'granularity': 'M1',
 'candles': [{'complete': True,
   'volume': 6,
   'time': '2019-03-01T04:19:00.000000000Z',
   'mid': {'o': '111.674', 'h': '111.676', 'l': '111.674', 'c': '111.676'}},
  {'complete': True,
   'volume': 6,
   'time': '2019-03-01T04:21:00.000000000Z',
   'mid': {'o': '111.674', 'h': '111.674', 'l': '111.672', 'c': '111.672'}},
  {'complete': True,
   'volume': 2,
   'time': '2019-03-01T04:22:00.000000000Z',
   'mid': {'o': '111.674', 'h': '111.676', 'l': '111.674', 'c': '111.676'}},
  {'complete': True,
   'volume': 4,
   'time': '2019-03-01T04:23:00.000000000Z',
   'mid': {'o': '111.674', 'h': '111.674', 'l': '111.667', 'c': '111.667'}},
  {'complete': True,
   'volume': 1,
   'time': '2019-03-01T04:24:00.000000000Z',
   'mid': {'o': '111.670', 'h': '111.670', 'l': '111.670', 'c': '111.670'}},
  {'complete': True,
   'volume': 2,
   'time': '2019-03-01T04:25:00.000000000Z',
   'mid': {'o': '111.672', 'h': '111.674', 'l': '111.672', 'c': 

In [127]:
# dataとしてリストへ過去レートを変換
data = []
for raw in r.response['candles']:
    data.append([raw['time'], raw['volume'], raw['mid']['o'], raw['mid']['h'], raw['mid']['l'], raw['mid']['c']])


In [136]:
# リストをPandasのDataFrame形式へ変換
df = pd.DataFrame(data)
df.columns = ['time', 'volume', 'open', 'high', 'low', 'close']
# df = df.set_index('time')
df.head() #無くても可

,time,volume,open,high,low,close
0,2019-03-01T04:19:00.000000000Z,6,111.674,111.676,111.674,111.676
1,2019-03-01T04:21:00.000000000Z,6,111.674,111.674,111.672,111.672
2,2019-03-01T04:22:00.000000000Z,2,111.674,111.676,111.674,111.676
3,2019-03-01T04:23:00.000000000Z,4,111.674,111.674,111.667,111.667
4,2019-03-01T04:24:00.000000000Z,1,111.670,111.670,111.670,111.670


In [137]:
# # date方がストリングで扱いづらいので、綺麗にする

# #日付の調整する
# #文字列 -> datetime
# def iso_jp(iso):
#     date = None
#     try:
#         date = datetime.strptime(iso, '%Y-%m-%dT%H:%M:%S.%fZ')
#         date = pytz.utc.localize(date).astimezone(pytz.timezone("Asia/Tokyo"))
#     except ValueError:
#         try:
#             date = datetime.strptime(iso, '%Y-%m-%dT%H:%M:%S.%f%z')
#             date = dt.astimezone(pytz.timezone("Asia/Tokyo"))
#         except ValueError:
#             pass
#     return date

In [138]:
# 上の関数を実行
df.head()

df['time'] = df['time'].str[:19]
df['time'] = datetime.strptime(df['time'], '%Y-%m-%dT%H:%M:%S.%fZ')
# DataFrameからCSVへ書き出し (easy)
# df.to_csv('api-usdjpy-M1.csv')

TypeError: strptime() argument 1 must be str, not Series

In [139]:
df.head()

,time,volume,open,high,low,close
0,2019-03-01T04:19:00,6,111.674,111.676,111.674,111.676
1,2019-03-01T04:21:00,6,111.674,111.674,111.672,111.672
2,2019-03-01T04:22:00,2,111.674,111.676,111.674,111.676
3,2019-03-01T04:23:00,4,111.674,111.674,111.667,111.667
4,2019-03-01T04:24:00,1,111.670,111.670,111.670,111.670
